<a href="https://colab.research.google.com/github/syedmahmoodiagents/Agents/blob/main/RAG_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 22.6 MB/s eta 0:00:00


In [2]:
!pip install -qU langchain langchain-community langchain-text-splitters --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
!pip install -qU langchain-openai langchain-faiss --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.0 MB/s eta 0:00:00


In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage

In [5]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [7]:
import os, getpass

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass() # can be replace by Ollama settings

··········


In [9]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [11]:
texts = [
    "LangChain helps developers build LLM applications.",
    "FAISS is used for vector similarity search.",
    "Chat history must be manually maintained in LangChain 1.1.",
    "Retrievers are used in RAG pipelines.",
    "OpenAI embeddings create vector representations."
]

In [12]:
embeddings = OpenAIEmbeddings()

In [13]:
db = FAISS.from_texts(texts, embeddings)

In [14]:
retriever = db.as_retriever()

In [15]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Use the retrieved context to answer the user."),
    ("human", "{question}\n\nContext:\n{context}")
])

In [16]:
chain = prompt | llm

In [17]:
history = []   # store HumanMessage + AIMessage objects

In [18]:
def ask(question):
    global history

    # Retrieve context documents
    docs = retriever.invoke(question)
    context = "\n".join([d.page_content for d in docs])

    # Add human message to history
    history.append(HumanMessage(content=question))

    # Run the RAG chain
    response = chain.invoke({
        "context": context,
        "messages": history,
        "question": question
    })

    # Add AI reply to history
    history.append(AIMessage(content=response.content))

    return response.content

In [10]:
print("User: What is FAISS?")
print("AI:", ask("What is FAISS?"))

print("\nUser: What did I ask earlier?")
print("AI:", ask("What did I ask earlier?"))

print("\nUser: How does LangChain handle memory?")
print("AI:", ask("How does LangChain handle memory?"))



User: What is FAISS?
AI: FAISS, which stands for Facebook AI Similarity Search, is a library designed for efficient vector similarity search. It is widely used to find similar items or perform nearest neighbor searches within large datasets of vector embeddings. This is particularly useful in applications like recommender systems or search features, where embeddings represent items, texts, or images in a high-dimensional space. In the context you provided, FAISS can be employed with OpenAI embeddings to perform similarity searches in retrieval-augmented generation (RAG) pipelines, enhancing the ability to fetch relevant information based on vector representations.

User: What did I ask earlier?
AI: I'm sorry, but I don't have access to previous interactions or the specific questions you may have asked earlier. How can I assist you today?

User: How does LangChain handle memory?
AI: LangChain handles memory by requiring developers to manually maintain chat history in version 1.1. This m

# One more example

In [ ]:
docs = ["LangChain 1.x has changed imports. ConversationalRetrievalChain is deprecated."]

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = splitter.create_documents(docs)

In [ ]:
emb = OpenAIEmbeddings()
vectordb = FAISS.from_documents(chunks, emb)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
prompt = ChatPromptTemplate.from_template("""
You are an assistant that answers based on retrieved context.

Context:
{context}

Question:
{question}

Answer:
""")

In [ ]:
def get_answer(query):
    context_docs = retriever.invoke(query)
    context = "\n\n".join([d.page_content for d in context_docs])

    chain = prompt | llm
    return chain.invoke({"context": context, "question": query})

In [ ]:
print(get_answer("What happened to ConversationalRetrievalChain?"))


content="In LangChain 1.x, the `ConversationalRetrievalChain` has been deprecated. This likely means that the functionality provided by `ConversationalRetrievalChain` has either been replaced by new components or incorporated into a different module or class structure. It's common for libraries to deprecate certain components as they evolve and improve their architecture to offer better modularity, performance, or usability. Users are typically encouraged to use the updated or new components as specified in the latest version of the library's documentation." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 51, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e

# Using fromMessages

In [19]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{question}"),
    ("human", "Relevant documents: {context}"),
])

In [20]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [21]:
def conversational_rag(question, chat_history):
    docs = retriever.get_relevant_documents(question)
    context = format_docs(docs)

    messages = [
        *chat_history,
        HumanMessage(content=prompt.format(question=question, context=context).to_string())
    ]

    response = llm(messages)
    return response.content


In [22]:

def get_answer(query):
    context_docs = retriever.invoke(query)
    context = "\n\n".join([d.page_content for d in context_docs])

    chain = prompt | llm
    return chain.invoke({"context": context, "question": query})


In [23]:
print(get_answer("What happened to ConversationalRetrievalChain?"))

content='ConversationalRetrievalChain is part of LangChain, which is a framework designed to help developers create applications using large language models (LLMs). In the context of LangChain, the ConversationalRetrievalChain typically enables the ability to perform retrieval-augmented generation (RAG) by allowing models to access relevant documents based on user queries and incorporate this information into the conversation flow.\n\nAs for the specifics of what happened to ConversationalRetrievalChain, it likely refers to changes or updates made to the functionality or implementation within LangChain, especially if there was a newer version released after 1.1. In this version, manual chat history maintenance is required, indicating that developers need to manage the context and previous interactions themselves. \n\nIf you are looking for further details on specific updates or features introduced after LangChain 1.1, checking the LangChain documentation or release notes would provide 